In [1]:
import recordlinkage as rl
import pandas as pd
import os
import random as ran
from IPython.display import clear_output


In [12]:
schema_path = "./schema_matching/csv/schema_final.csv"

In [13]:
schema = pd.read_csv(schema_path, index_col=0)
schema = schema.reset_index(drop=True)

# Preprocessamento

In [4]:
from recordlinkage.preprocessing import *

## Cleaning dei dati ...

In [14]:
schema_cleaned = schema

In [6]:
for i in range(len(schema.index)):
    row = schema.iloc[i]
    row_series = row.squeeze()
    nome = pd.Series(row["name"])
    try:
        nome_cleaned = clean(nome, lowercase=True, replace_by_none='', replace_by_whitespace='', strip_accents='unicode', remove_brackets=False, encoding='utf-8', decode_error='strict')
    except:
        print(nome)
    row_series_cleaned = clean(row_series, lowercase=True, replace_by_none=r'[^ \\.\\-\\_A-Za-z0-9$€£¥₩₽₹฿₪¤¢₺₱]+', replace_by_whitespace=r'[\\-\\_]', strip_accents=None, remove_brackets=False, encoding='utf-8', decode_error='strict')
    schema_cleaned.iloc[i] = row_series_cleaned
    schema_cleaned.iloc[i]["name"] = nome_cleaned[0]
print(schema_cleaned)

                                            name        country market cap  \
0                             berkshire hathaway  united states        NaN   
1                                           icbc          china        NaN   
2       saudi arabian oil company (saudi aramco)   saudi arabia        NaN   
3                                 jpmorgan chase  united states        NaN   
4                        china construction bank          china        NaN   
...                                          ...            ...        ...   
188552                                   gazprom         russia        NaN   
188553                                   rosseti         russia        NaN   
188554                                 nornickel         russia        NaN   
188555                                 severstal         russia        NaN   
188556                                      ozon            usa        NaN   

       founded year employees                        industry s

# Occorrenze token per riga

In [7]:
# value_occurence(schema)

# Blocking

In [8]:
indexer = rl.Index()
indexer.block(left_on='name', right_on='name')
candidate_links = indexer.index(schema_cleaned)
print((candidate_links))

MultiIndex([(  2966,      0),
            ( 12966,      0),
            ( 12966,   2966),
            ( 30538,      0),
            ( 30538,   2966),
            ( 30538,  12966),
            ( 34398,      0),
            ( 34398,   2966),
            ( 34398,  12966),
            ( 34398,  30538),
            ...
            (176795, 176576),
            (176851, 176580),
            (176729, 176591),
            (176697, 176602),
            (176751, 176606),
            (177007, 176616),
            (176863, 176642),
            (176934, 176671),
            (176984, 176796),
            (177004, 176877)],
           length=549188)


# Labeling

In [11]:
keys = ['name', 'country', 'market cap', 'founded year', 'employees', 'industry', 'sector',
     'ceo', 'revenue', 'stock', 'share price', 'city', 'address', 'website']

clear = lambda: os.system('clear')

In [222]:
n_match = 500
choices = ran.choices(candidate_links, k=n_match)

lKeys = {key : f"l_{key}" for key in keys}
rKeys = {key : f"r_{key}" for key in keys}

idxKeys = ["id_1", "id_2"]

choices_column = [*idxKeys, *lKeys.values(), *rKeys.values()]
choices_df = pd.DataFrame(columns=choices_column)
for choice in choices:
    lRow = schema_cleaned.iloc[[choice[0]]]
    rRow = schema_cleaned.iloc[[choice[1]]]

    lRow = lRow.rename(columns=lKeys).reset_index(drop=True)
    rRow = rRow.rename(columns=rKeys).reset_index(drop=True)

    idxs = pd.DataFrame({idxKeys[0]: [choice[0]], idxKeys[1]: [choice[1]]})
    row = pd.concat([idxs, lRow, rRow], axis=1)
    choices_df = pd.concat([choices_df, row], axis=0)

# choices_df.reset_index(inplace=True)
choices_df.set_index(idxKeys)

,,l_name,l_country,l_market cap,l_founded year,l_employees,l_industry,l_sector,l_ceo,l_revenue,l_stock,...,r_employees,r_industry,r_sector,r_ceo,r_revenue,r_stock,r_share price,r_city,r_address,r_website
id_1,id_2,,,,,,,,,,,,,,,,,,,,,
65823,28798,Shin-etsu Chemical Co Ltd,NaN,¥7.770 Trillion,NaN,"24,629",NaN,NaN,NaN,NaN,NaN,...,"24,954",Chemicals,NaN,NaN,$18.5B,NaN,NaN,NaN,"Asahi Seimei Otemachi Bldg., 2-6-1, Ote-Machi,...",www.shinetsu.co.jp
182366,80742,Mitsubishi Estate,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$13.62,NaN,NaN,NaN
31475,31311,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harju maakond Tallinn ...,NaN
47431,39330,JBM Auto,india,$0.80 B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$6.76,NaN,NaN,NaN
56885,4914,SHANDONG HUALU-HENGSHENG CHEMICAL,NaN,NaN,NaN,3921,NaN,NaN,NaN,NaN,NaN,...,NaN,"['Basic Materials', 'Chemicals', 'Fertilizers'...",NaN,Not found,"4,181 Million USD",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95604,63819,Network18 Media & Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53759,46123,Repsol,NaN,NaN,1987; 35 years ago (1987),NaN,Petroleum,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$16.45,NaN,NaN,NaN
137078,90194,BANKINTER,Spain,5.288 Billion USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"8,668",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.bankinter.com/


In [205]:
choices_path = "./schema_matching/csv/choices.csv"
choices_df.to_csv(choices_path, index=None)


In [223]:
choices_df = pd.read_csv(choices_path, index_col=[0, 1])
choices_df

,,l_name,l_country,l_market cap,l_founded year,l_employees,l_industry,l_sector,l_ceo,l_revenue,l_stock,...,r_employees,r_industry,r_sector,r_ceo,r_revenue,r_stock,r_share price,r_city,r_address,r_website
id_1,id_2,,,,,,,,,,,,,,,,,,,,,
113518,83236,Computer Programs and Systems,NaN,$0.41 B,NaN,2.4,NaN,NaN,NaN,$0.31 B,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$28.11,NaN,NaN,NaN
130804,30036,sse plc,uk,NaN,NaN,"12,489",NaN,utilities,Alistair Phillips-Davies,"10,139 Million USD",NaN,...,"11,865",Utilities—Diversified,Utilities,Mr. Alistair Phillips-Davies,"£6,826.4 million (2021)",NaN,NaN,NaN,NaN,NaN
145968,21992,STRABAG,Austria,$3.8 B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Engineering & Construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177617,98879,Vivendi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"42,800",NaN,NaN,Arnaud de Puyfontaine,NaN,NaN,NaN,NaN,NaN,https://www.vivendi.com
31576,30868,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harju maakond Tallinn ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141343,10682,TIANRUN CRANKSHAFT,China,1.162 Billion USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"['Consumer Cyclical', 'Automobile', 'Automobil...",NaN,Not found,Not found,NaN,NaN,NaN,NaN,NaN
47949,38717,S&T Bancorp,usa,$1.30 B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$33.96,NaN,NaN,NaN
84638,21525,Diversey,UK,$1.76 B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Chemicals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [237]:
golden_path = "./schema_matching/csv/golden_links.csv"
try: 
    golden = pd.read_csv(golden_path, index_col=[0, 1])
except:
    golden = pd.DataFrame(columns=[*choices_column, "label"])
    golden.set_index(idxKeys)

In [238]:
golden

,id_1,id_2,l_name,l_country,l_market cap,l_founded year,l_employees,l_industry,l_sector,l_ceo,...,r_industry,r_sector,r_ceo,r_revenue,r_stock,r_share price,r_city,r_address,r_website,label


In [239]:

for idxs, choice in choices_df.iterrows():
    print("I due record appartengono alla stessa entità?")
    print("y -> sì;  n -> no;  q -> esci")
    choice = choice.to_frame().T.reset_index(drop=True)

    display(choice[lKeys.values()])
    display(choice[rKeys.values()])

    stop = False
    while(True):
        inp = input()
        
        if(inp == 'q'):
            stop = True
            break
        
        if inp not in ["y", "n"]:
            continue


        idx = pd.DataFrame({idxKeys[0]: [idxs[0]], idxKeys[1]: [idxs[1]]})
        label = pd.DataFrame({"label" : [int(inp == "y")]})

        row = pd.concat([idx, choice, label], axis=1)
        golden = pd.concat([golden, row], axis=0)
        
        choices_df.drop(axis=0, labels=idxs, inplace=True)
        break
    
    if(stop):
        break
    clear_output()


I due record appartengono alla stessa entità?
y -> sì;  n -> no;  q -> esci


,l_name,l_country,l_market cap,l_founded year,l_employees,l_industry,l_sector,l_ceo,l_revenue,l_stock,l_share price,l_city,l_address,l_website
0,Doosan Bobcat,S. Korea,$2.75 B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$27.51,NaN,NaN,NaN


,r_name,r_country,r_market cap,r_founded year,r_employees,r_industry,r_sector,r_ceo,r_revenue,r_stock,r_share price,r_city,r_address,r_website
0,DOOSAN BOBCAT,NaN,NaN,NaN,"2,601",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.doosanbobcat.com/


In [240]:
golden

,id_1,id_2,l_name,l_country,l_market cap,l_founded year,l_employees,l_industry,l_sector,l_ceo,...,r_industry,r_sector,r_ceo,r_revenue,r_stock,r_share price,r_city,r_address,r_website,label
0,177617,98879,Vivendi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Arnaud de Puyfontaine,NaN,NaN,NaN,NaN,NaN,https://www.vivendi.com,1
0,31576,30868,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harju maakond Tallinn ...,NaN,1


In [241]:
choices_df

,,l_name,l_country,l_market cap,l_founded year,l_employees,l_industry,l_sector,l_ceo,l_revenue,l_stock,...,r_employees,r_industry,r_sector,r_ceo,r_revenue,r_stock,r_share price,r_city,r_address,r_website
id_1,id_2,,,,,,,,,,,,,,,,,,,,,
167075,101896,Doosan Bobcat,S. Korea,$2.75 B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"2,601",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.doosanbobcat.com/
167668,94718,Freehold Royalties,Canada,$1.78 B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.freeholdroyalties.com/
87554,1504,Verisign,NaN,NaN,NaN,902,NaN,NaN,D. James Bidzos,NaN,NaN,...,904,IT Software & Services,NaN,D. James Bidzos,$1.3B,NaN,NaN,NaN,NaN,NaN
43047,29696,CITIC Securities Company Ltd,China,$40.17 B,NaN,"11,380",Capital Markets,Financials,NaN,NaN,600030,...,"11,380",Capital Markets,Financials,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55292,35000,ADIDAS,NaN,NaN,NaN,62285,NaN,NaN,Kasper Rorsted,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$162.60,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141343,10682,TIANRUN CRANKSHAFT,China,1.162 Billion USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"['Consumer Cyclical', 'Automobile', 'Automobil...",NaN,Not found,Not found,NaN,NaN,NaN,NaN,NaN
47949,38717,S&T Bancorp,usa,$1.30 B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,$33.96,NaN,NaN,NaN
84638,21525,Diversey,UK,$1.76 B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Chemicals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [242]:
choices_df.to_csv(choices_path)
# TODO: da sistemare problema con gli indici al
#       salvataggio dopo il primo imoprt di golden
golden.to_csv(golden_path, index=None)

In [214]:
print(len(candidate_links)/len(schema_cleaned)**2)

1.544670027008048e-05


# Pre-Labeling

In [ ]:
compare_cl = rl.Compare()
compare_cl.string("name", "name", threshold=0.85, label="name")
compare_cl.string("country", "country", label="country")
compare_cl.string("founded year", "founded year", label="founded year")
compare_cl.string("industry", "industry", label="industry")
compare_cl.string("sector", "sector", label="sector")
compare_cl.string("address", "address", label="address")
compare_cl.string("city", "city", label="city")
compare_cl.string("ceo", "ceo", label="ceo")
features = compare_cl.compute(candidate_links, schema, schema)

In [ ]:
features

In [ ]:
features.describe()

In [ ]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
record_linked = features[features.sum(axis=1) > 3]

In [ ]:
print(record_linked)

# Classification

In [ ]:
ECM = rl.ECMClassifier(binarize=0.6)
record_linked.index.names = (['id1', 'id2'])
print(len(record_linked.index))
result = ECM.fit_predict(record_linked)
print(len(result))